In [13]:
import scrapy
from urllib.parse import urljoin


class RatingSpider(scrapy.Spider):
    name = "rate"
    start_urls = ["https://www.imdb.com/title/tt4695012/reviews?ref_=tt_ql_3"]

    def parse(self, response):
        ratings = response.xpath("//div[@class='ipl-ratings-bar']//span[@class='rating-other-user-rating']//span[not(contains(@class, 'point-scale'))]/text()").getall()
        texts = response.xpath("//div[@class='text show-more__control']/text()").getall()
        result_data = []
        for i in range(len(ratings)):
            row = {
                "ratings": int(ratings[i]),
                "review_text": texts[i]
            }
            result_data.append(row)
            print(json.dumps(row))

        key = response.css("div.load-more-data::attr(data-key)").get()
        orig_url = response.meta.get('orig_url', response.url)
        next_url = urljoin(orig_url, "reviews/_ajax?paginationKey={}".format(key))
        if key:
            yield scrapy.Request(next_url, meta={'orig_url': orig_url})
            
        with open("test.html", 'wb') as f:
            f.write(response.body)

In [4]:
rs = RatingSpider()

In [11]:
result = []
for i in rs.start_requests():
    result.append(i)

In [12]:
result

[<GET https://www.imdb.com/title/tt4695012/reviews?ref_=tt_ql_3>]